### 第1セル

In [13]:
# 質問文
query = "富士山は日本で何番目に高い山ですか？"

### 第2セル

In [14]:
# 検索対象

texts = [
    "日本で一番高い山は富士山です。",
    "富士山は日本で最も大きい山です。",
    "Mt. Fuji is the highest mountain in Japan.",
    "富士山プリンは日本で一番おいしい。",
    "エベレストは世界で一番高い山です。",
    "アメリカには自由の女神があります。"
]

### 第3セル

In [20]:
# キーワード検索

import numpy as np
import faiss
import torch
from transformers import AutoTokenizer, AutoModel
from janome.tokenizer import Tokenizer as JanomeTokenizer
from rank_bm25 import BM25Okapi

#トークンに分ける
janome_tokenizer = JanomeTokenizer()
def tokenize(text):
    allowed_pos = {"名詞", "動詞", "形容詞", "副詞"}
    tokens = []
    for token in janome_tokenizer.tokenize(text):
        if token.part_of_speech.split(',')[0] in allowed_pos:
            tokens.append(token.surface)
    return tokens

#トークンで分ける
tokenized_docs = [tokenize(doc) for doc in texts]
keyword_model = BM25Okapi(tokenized_docs)
tokenized_query = tokenize(query)
keyword_scores = keyword_model.get_scores(tokenized_query)

# キーワード検索の順位付け
keyword_ranked = sorted(zip(keyword_scores, texts), reverse=True)

# print("クエリの形態素解析結果:", tokenized_query)
# print("テキストの形態素解析結果：", tokenized_docs)

# ベクトル検索

MODEL_NAME = "./bge-m3"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

def encode(texts):
    with torch.no_grad():
        inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to(DEVICE)
        outputs = model(**inputs)
        
        # mean_pooling + 即座正規化
        token_embeddings = outputs.last_hidden_state
        mask = inputs["attention_mask"].unsqueeze(-1).expand(token_embeddings.size()).float()
        embeddings = torch.nn.functional.normalize(
            (token_embeddings * mask).sum(dim=1) / mask.sum(dim=1).clamp(min=1e-9), 
            p=2, dim=1
        )
        
        return embeddings.cpu().numpy()

doc_embeddings = encode(texts)
query_embedding = encode([query])

# 既に正規化済みなのでそのまま使用
index = faiss.IndexFlatIP(doc_embeddings.shape[1])
index.add(doc_embeddings)
similarities, indices = index.search(query_embedding, 6)

print("=== キーワード検索上位5件 ===")
for rank, (score, doc) in enumerate(keyword_ranked[:6], 1):
    print(f"{rank}:{doc} (score={score:.2f})")

print("\n=== ベクトル検索上位5件（類似度） ===")
for rank, idx in enumerate(indices[0]):
    print(f"{rank+1}:{texts[idx]} (similarity={similarities[0][rank]:.4f})")

=== キーワード検索上位5件 ===
1:日本で一番高い山は富士山です。 (score=0.62)
2:エベレストは世界で一番高い山です。 (score=0.62)
3:富士山プリンは日本で一番おいしい。 (score=0.00)
4:富士山は日本で最も大きい山です。 (score=0.00)
5:アメリカには自由の女神があります。 (score=0.00)
6:Mt. Fuji is the highest mountain in Japan. (score=0.00)

=== ベクトル検索上位5件（類似度） ===
1:日本で一番高い山は富士山です。 (similarity=0.9052)
2:富士山は日本で最も大きい山です。 (similarity=0.9008)
3:Mt. Fuji is the highest mountain in Japan. (similarity=0.8347)
4:富士山プリンは日本で一番おいしい。 (similarity=0.7952)
5:エベレストは世界で一番高い山です。 (similarity=0.7523)
6:アメリカには自由の女神があります。 (similarity=0.6102)


### 第4セル

In [ ]:
# リランキング

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np

RERANK_MODEL_DIR = "./japanese-bge-reranker-v2-m3-v1"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(RERANK_MODEL_DIR)
model = AutoModelForSequenceClassification.from_pretrained(RERANK_MODEL_DIR).to(DEVICE)

# キーワード検索文書リスト（スコアを除いてテキストのみ）
keyword_docs_texts = [doc for score, doc in keyword_ranked]

def rerank(query, candidate_docs):
    inputs = tokenizer(
        [query] * len(candidate_docs),
        candidate_docs,
        padding=True,
        truncation=True,
        return_tensors="pt"
    ).to(DEVICE)

    with torch.no_grad():
        outputs = model(**inputs)
        scores = outputs.logits.squeeze(-1).cpu().numpy()
    
    rerank_order = np.argsort(scores)[::-1]
    return rerank_order, scores

rerank_order, rerank_scores = rerank(query, keyword_docs_texts)

print("=== リランキング結果 ===")
for i, idx in enumerate(rerank_order):
    print(f"{i+1}. {keyword_docs_texts[idx]} (score={rerank_scores[idx]:.4f})")

=== リランキング結果 ===
1. 日本で一番高い山は富士山です。 (score=6.0030)
2. Mt. Fuji is the highest mountain in Japan. (score=5.3655)
3. 富士山は日本で最も大きい山です。 (score=2.2264)
4. エベレストは世界で一番高い山です。 (score=-2.7905)
5. 富士山プリンは日本で一番おいしい。 (score=-5.9260)
6. アメリカには自由の女神があります。 (score=-10.6326)
